# Compare Flights example

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

## Importing libraries

In [ ]:
from rocketpy.plots.compare import CompareFlights
from rocketpy import Environment, Flight, Rocket, SolidMotor
from datetime import datetime, timedelta

## Creating the Flight objects

First, let's create the environment, motor and rocket objects.

In [ ]:
after_tomorrow = datetime.now() + timedelta(days=2)
env = Environment(latitude=-23, longitude=-49, date=after_tomorrow)
env.set_atmospheric_model(type="Forecast", file="GFS")

Pro75M1670 = SolidMotor(
    thrust_source="../../data/motors/Cesaroni_M1670.eng",
    burn_time=3.9,
    grains_center_of_mass_position=-0.85704,
    grain_number=5,
    grain_separation=5 / 1000,
    grain_density=1815,
    grain_outer_radius=33 / 1000,
    grain_initial_inner_radius=15 / 1000,
    grain_initial_height=120 / 1000,
    nozzle_radius=33 / 1000,
    throat_radius=11 / 1000,
    interpolation_method="linear",
    nozzle_position=-1.255,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

calisto = Rocket(
    radius=127 / 2000,
    mass=19.197 - 2.956,
    inertia_i=6.60,
    inertia_z=0.0351,
    power_off_drag="../../data/calisto/powerOffDragCurve.csv",
    power_on_drag="../../data/calisto/powerOnDragCurve.csv",
    center_of_dry_mass_position=0,
    coordinate_system_orientation="tail_to_nose",
)

calisto.set_rail_buttons(0.2, -0.5)

calisto.add_motor(Pro75M1670, position=-1.255)

nosecone = calisto.add_nose(
    length=0.55829, kind="vonKarman", position=0.71971 + 0.55829
)

fin_set = calisto.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.040,
    span=0.100,
    position=-1.04956,
    cant_angle=0,
    radius=None,
    airfoil=None,
)

tail = calisto.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)


def drogue_trigger(p, h, y):
    # p = pressure considering parachute noise signal
    # h = height above ground level considering parachute noise signal
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]

    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def main_trigger(p, h, y):
    # p = pressure considering parachute noise signal
    # h = height above ground level considering parachute noise signal
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]

    # activate main when vz < 0 m/s and z < 800 m
    return True if y[5] < 0 and h < 800 else False


Main = calisto.add_parachute(
    "Main",
    cd_s=10.0,
    trigger=main_trigger,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue = calisto.add_parachute(
    "Drogue",
    cd_s=1.0,
    trigger=drogue_trigger,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Now we can create different flights varying the launch angle and the rail 
inclination:

In [ ]:
inclinations = [85, 75, 60]
headings = [90, 135]
flights = []

for heading in headings:
    for inclination in inclinations:
        flight = Flight(
            environment=env,
            rocket=calisto,
            rail_length=5
            inclination=inclination,
            heading=heading,
            name=f"Incl {inclination} Head {heading}",
        )
        flights.append(flight)


In [ ]:
print("Number of flights: ", len(flights))

## Start the comparison

In [ ]:
comparison = CompareFlights(flights)

### Linear Kinematics

In [ ]:
comparison.trajectories_3d(legend=True)

In [ ]:
comparison.positions()

In [ ]:
comparison.trajectories_2d(plane="xy", legend=True)

In [ ]:
comparison.velocities()

In [ ]:
comparison.stream_velocities()

In [ ]:
comparison.accelerations()

### Angular kinematics

In [ ]:
comparison.angular_velocities()

In [ ]:
comparison.angular_accelerations()

### Attitudes

In [ ]:
comparison.attitude_angles()

In [ ]:
comparison.euler_angles()

In [ ]:
comparison.quaternions()

In [ ]:
comparison.angles_of_attack()

### Forces and Moments

In [ ]:
comparison.aerodynamic_forces()

In [ ]:
comparison.aerodynamic_moments()

In [ ]:
# comparison.rail_buttons_forces() # Currently not working!

### Energies and Power

In [ ]:
comparison.fluid_mechanics()

In [ ]:
comparison.energies()

In [ ]:
comparison.powers()

## Plotting everything together

Alternatively, we can plot all the results together by calling one simple 
method:

In [ ]:
comparison.all()